In [ ]:
from numpy import loadtxt, pi
path = loadtxt(r"C:\Users\Srinivas\Documents\GitHub\New_world\robot_trajectory.txt")
poses = []
for i in range(len(path)):
    if path[i][3]!=0:
        if path[i][3] == 1:
            poses.append([path[i][0],path[i][1],path[i-1][2],pi])
        elif path[i][3] == 3:
            poses.append([path[i][0],path[i][1],path[i-1][2],0])
        elif path[i][3] == 2:
            poses.append([path[i][0],path[i][1],path[i-1][2],3*pi/2])
        elif path[i][3] == 4:
            poses.append([path[i][0],path[i][1],path[i-1][2],pi/2])

lposes = len(poses)

In [ ]:
from math import tan, atan, sqrt, cos, sin
from numpy import eye, diag, array, exp, ceil, floor, dot, arange
from numpy.linalg import inv, det
from random import gauss, randint, uniform
import copy
import sys

class Particle:
    def __init__(self,pose):
        self.pose = pose
        self.landmark_poses = []
        self.landmark_covariances = []
        self.wall_list = [] # Stores the histogram from the tree w
        self.var = 0
        self.length = 480
#         self.w = [['0'],['90'],['180'],['270']] # Landmark as histograms stored by each particle
        self.w = []
    
    def number_of_landmarks(self):
        return len(self.w)
    
    @staticmethod
    def motion(pose, control, WB, dt):  
        angle = (control[1]+pose[2])%(2*pi)
        return ([x+y for x,y in zip(pose,[control[0]*dt*cos(angle),
                                          control[0]*dt*sin(angle),
                                          control[0]*dt*sin(control[1])/WB])])
    
    def move(self,control,WB,dt,pose):
        self.pose = pose
        
    def expected_measurement(self,pose,landmark):   # Not considering the other component  
        return array([landmark[0],tan(landmark[1]-pose[2])])      
    
    def measurement_correspondence(self,pose,measurement,number_of_landmarks,Qt_measurement_covariance):
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence(measurement,i,Qt_measurement_covariance))
        return likelihoods
    
    @staticmethod
    def dh_dlandmark(pose,landmark):
        return array([[1,0],[0,-1/(cos(pose[2]-landmark[2])**2)]]) # H - Direct odometry measurement and collision
    
    def H_and_Ql_from_measurement(self,landmark,Qt_measurement_covariance,i):
        
        H = self.dh_dlandmark(self.pose,landmark)
        Ql = dot(dot(H,self.landmark_covariances[i]),H.T) + Qt_measurement_covariance
        
        return (H, Ql)
        
    def landmark_correspondence_likelihood(self,measurement,landmark_number,Qt):
        # For a given measurment and a landmark number, it returns a suitable likelihood value of the correspondence

        landmark = self.w[landmark_number]
        num_wall = len(landmark)-1
        likelihood=[]
        for i in arange(1,num_wall,1):
            
            phi = landmark[i][1]
            theta = self.pose[2]

            # Prune the landmark which are not in the robot's frame of collision
            if phi == pi/2 * floor(theta/(pi/2)) or phi == pi/2 * ceil(theta/(pi/2)): 
                if landmark[i][1] % pi == 0:
                    # Check the x-value correspondence
                    zhat = self.expected_measurement(self.pose,landmark[i])
                    H, Ql = self.H_and_Ql_from_measurement(landmark[i],diag([Qt[0][0],Qt[2][2]]),landmark_number)
                    dz = array([measurement[0],measurement[2]])-zhat
                else:
                    # Check the y-value correspondence
                    zhat = self.expected_measurement(self.pose,landmark[i])
                    H, Ql = self.H_and_Ql_from_measurement(landmark[i],diag([Qt[1][1],Qt[2][2]]),landmark_number)
                    dz = array([measurement[1],measurement[2]])-zhat
                
                # Compute likelihood
                sqrtdetQl = sqrt(det(Ql))
                normal = 1 / (2*pi*sqrtdetQl)
                l = normal * exp(-0.5*dot(dot(dz.T,inv(Ql)),dz))            
            else: 
                l = 0
            likelihood.append(l)
        return max(likelihood)
    
    def dis_Bayes(self,wall_num, index, index_sttdev): # The landmark, the position of measurement of 
        # that landmark and stddev(covariance)
        
        for i in range(index_stddev):
            # Corridor width=120, Standard deviation=11, Creating a triangular distribution of width=2sigma
            b=1.5-i*(0.5/index_stddev) 
            if j-i >= 0: 
                self.wall_list[wall_num][index-i]*=b
            if j+i <=l-1:
                self.wall_list[wall_num][index+i]*=b

            self.wall_list[wall_num] = [i*5/sum(wall_list[wall_num]) for i in a] # Scaling parameter=5
    
    def initialize_new_landmark(self,measurement,Qt, l):
        # Measurement to be modified (Orientations to 90 degrees)
        self.w.insert(self.var,[self.var])        
        orientation = self.pose[2]+atan(measurement[2])
        
        if orientation % pi == 0: # Need to encode x and theta component in Sigma
            self.w[self.var].append([poses[0], orientation])
            self.wall_list.append([1/l]*l)
            self.dis_Bayes(self.var,int(poses[1]),20)
            
            Hinv = array([[1,0],[0,1/(1+measurement[2]**2)]])
            Qtx = np.diag([Qt[0][0],Qt[2][2]])
            Sigma = dot(dot(Hinv,Qtx),Hinv.T)
            self.landmark_covariances.append(Sigma)
        else:
            self.w[self.var].append([poses[1], orientation])
            self.wall_list.append([1/l]*l)
            self.dis_Bayes(self.var,int(poses[0]),20)   
            
            Hinv = array([[1,0],[0,1/(1+measurement[2]**2)]])
            Qty = np.diag([Qt[1][1],Qt[2][2]]) # y component and orientation
            Sigma = dot(dot(Hinv,Qty),Hinv.T)
            self.landmark_covariances.append(Sigma)
            
        self.var = self.var + 1
        
    def update_landmark(self,measurement,landmark_number,Qt_measurement_covariance):
        
        for i in arange(1,len(self.w[landmark_number]),1):
            mu = self.w[landmark_number][i][0]
            Sigma = self.landmark_covariances[landmark_number]
            H, Ql = self.H_and_Ql_from_measurement(self.w[landmark_number][i],Qt_measurement_covariance,landmark_number)
        K = dot(dot(Sigma,H.T),inv(Ql))
        
        mu = mu + dot(K,(measurement - self.expected_measurement(self.pose,mu)))
        Sigma = dot((eye(3) - K*H),Sigma)
        
        return (mu,Sigma)
    
    def update_particle(self,measurement,number_of_landmarks,minimum_correspondence_likelihood,Qt_measurement_covariance):
        likelihoods = []
        for i in range(number_of_landmarks):
            likelihoods.append(self.landmark_correspondence_likelihood(measurement,i,Qt_measurement_covariance))

        if not likelihoods or max(likelihoods) < minimum_correspondence_likelihood:
            self.initialize_new_landmark(measurement,Qt_measurement_covariance, self.length)
            return minimum_correspondence_likelihood
        
        else:
            lmax = max(likelihoods)
            lmax_index = likelihoods.index(lmax)
            mu,Sigma = self.update_landmark(measurement,lmax_index,Qt_measurement_covariance)
            self.landmark_poses[lmax_index] = mu
            self.landmark_covariances[lmax_index] = Sigma
            
            return lmax

class FastSLAM:
    def __init__(self,initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,
                 x_stddev,y_stddev,control_speed_factor,control_head_factor, sample_time):
        # Particles
        self.particles = initial_particles
        
        # Constants
        self.robot_width = robot_width
        self.minimum_correspondence_likelihood = minimum_correspondence_likelihood
        self.xstddev = xstddev
        self.ystddev = ystddev
        self.measurement_stddev = measurement_stddev
        self.control_speed_factor = control_speed_factor
        self.control_head_factor = control_head_factor
        self.dt = sample_time
        self.WB = robot_width
        
    def predict(self,pose,control):
        # Prediction step of FastSLAM

        speed, head = control
        speed_std = self.control_speed_factor * sqrt(speed) # To be modified
        head_std  = self.control_head_factor * sqrt(359 - head) # To be modified; mirror image of the speed deviation
        
        for p in self.particles:
            speed = gauss(speed,speed_std)
            head = gauss(head,head_std)
            p.move([speed,head],self.WB,self.dt,pose)
    
    def update_and_compute_weights(self,measurement):

        Qt_measurement_covariance = diag([self.xstddev ** 2, self.ystddev ** 2,self.measurement_stddev ** 2]) 
        weights = []
        for p in self.particles:
            number_of_landmarks = p.number_of_landmarks()
            weight = p.update_particle(measurement,number_of_landmarks,self.minimum_correspondence_likelihood,
                                            Qt_measurement_covariance)
            weights.append(weight)
            
        return weights
    
    def resample(self,weights):
        
        new_particles = []
        max_weight = max(weights)
        index = randint(0, len(self.particles)-1)
        offset = 0.0
        for i in xrange(len(self.particles)):
            offset += uniform(0, 2.0 * max_weight)
            while offset > weights[index]:
                offset -= weights[index]
                index = (index + 1) % len(weights)
            new_particles.append(copy.deepcopy(self.particles[index]))
            
        return new_particles
    
    def correct(self,measurement):
        
        weights = self.update_and_compute_weights(measurement)
        self.particles = self.resample(weights)
        
        return weights
        
if __name__ == '__main__':
    
    robot_width = 2
    sample_time = 0.01
    
    minimum_correspondence_likelihood = 1e-3
    xstddev = 0.001
    ystddev = 0.001
    measurement_stddev = 0.001
    
    control_speed_factor = 0.01
    control_head_factor = 0.01
    number_of_particles = 10
    
    start_state = poses[0][0:3]
    initial_particles = [copy.copy(Particle(start_state))
                         for _ in xrange(number_of_particles)]

    fs = FastSLAM(initial_particles,robot_width,minimum_correspondence_likelihood,measurement_stddev,xstddev,ystddev,
                 control_speed_factor,control_head_factor, sample_time)
    
    for i in xrange(lposes-1):
        # Correction step
        print "Iteration",i
        '''Use of entire trajectory for the correction step (Rao-Blackwellization)''' 
        w = fs.correct(array([poses[i][0],poses[i][1],tan(poses[i][3]-poses[i][2])])) # Odometry and collision values
        #Prediction
        fs.predict(poses[i+1][0:3],[100,poses[i+1][2]])

In [ ]:
import matplotlib.pyplot as plt
from time import sleep
from IPython import display

xl = []
yl = []
xt = []
yt = []

for i in xrange(len(fs.particles[0].landmark_poses)):
    xl.append(fs.particles[0].landmark_poses[i][0])
    yl.append(fs.particles[0].landmark_poses[i][1])
    xt.append(poses[i][0])
    yt.append(poses[i][1])

plt.plot(xl,yl,'o',color='b')
plt.plot(xt,yt,'o',color='r')
plt.show()
print "xl is",xl
print "yl is",yl
print "xt is",xt
print "yt is",yt

In [8]:
a=[]
f1=[]
a.insert(0,f1)
f1.append([111.5,0])
print a

[[[111.5, 0]]]
